In [17]:
import utils
import importlib
importlib.reload(utils)
import pandas as pd
import numpy as np

network_folder = '//teamwork.org.aalto.fi/T30814-MASSE/Juuso Heinonen/' # Täällä siis pitäisi olla ne datat saatavilla

In [18]:
visits = utils.read_visits_csv(network_folder, filenames={'visits': 'KÄYNNIT korjattu poiminta 7.8.2017_3.10.2018.csv', 'ICD': 'ICD10_luokitus.xlsx', 'ICPC': 'ICPC_luokitus.xlsx', 'ASL_ICD': 'ASL_ICD_10.csv', 'ASL_ICPC': 'ASL_ICPC2.csv', 'ASL_names': 'ASL_nimet.csv'},
                               include_cost=True, cost_filename=network_folder + 'Yksikkökustannukset.xlsx')
contacts = utils.read_contacts_csv(network_folder + 'Yhteydenotto poiminta 7.8.2017_3.10.2018.csv', include_cost=True, cost_filename=network_folder + 'Yksikkökustannukset.xlsx')
bookings = utils.read_bookings_csv(network_folder + 'Ajanvaraus sähköinen HTA poiminta 7.8.2017_3.10.2018.csv', include_cost=True, cost_filename=network_folder + 'Yksikkökustannukset.xlsx')

visits = utils.create_Klinik_visits_from_contacts(visits, contacts) # Tässä ajatuksena lisätä toisesta tiedostosta ilmeisesti puuttuvia Klinik käyntejä
visits = utils.add_user(visits, include_walk_in=True) # Lisää "user" luokittelun: Klinik, phone, walk-in, other. Jokainen potilas kuuluu yhteen noista ja siis ensimmäiseen mahdolliseen. Esim potilas jolla on sekä Klinik että puhelin HTA on Klinik potilas.

C:\Users\canye\.conda\envs\dippa\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [19]:
visits = utils.create_ASL_episodes(visits, bookings) # Lisää tämänhetkisen tavan tehdä noita episodeja. Episodin määrittää uniikisti "Patiend ID - Episode" yhdistelmä.

In [20]:
visits.head()

,Toimipisteen lyhenne,Käyntiluokan koodi,Vastaanottajan nimi,Potilaan lista käyntipäivänä,Time,Käyntiluokan nimi,Age,Sex,Patient ID,Käyntiryhmän koodi,...,Walk-in_user,User,orig_ASL,Booked ASL,Booking ASL,Episode,Booked Episode,Booking Episode,Urgency,Episode_ASL
180966,MYTAA,AAA1R,"SALOKARI,ESKO",31,2018-08-30,RESEPTIN UUSINTA TAI TODISTUS,41.0,True,65650023,ABA1A,...,False,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9153,MYTAA,AAA1P,"MANNINEN,TUIJA",31,2017-08-28,PERUSTYÖAJAN KÄYNTI LÄÄK.VAST.OTOLLA(AJANVARAUS),66.0,True,65650030,ABA1A,...,True,Phone,Other diseases of the musculoskeletal system,NaN,NaN,None,NaN,NaN,NaN,None
9710,MYTAA,AAA1O,"MANNINEN,TUIJA",31,2017-08-29,PAPERIKONSULTAATIO,66.0,True,65650030,ABA1A,...,True,Phone,NaN,NaN,NaN,None,NaN,NaN,NaN,None
14061,MYTAB,AAA1T,"MANNINEN,ANGELA",31,2017-09-06,PUHELINKONTAKTI PERUSTYÖAIKANA,66.0,True,65650030,ABA1A,...,True,Phone,NaN,NaN,NaN,None,NaN,NaN,NaN,None
14872,MYTAA,AAA1T,"MANNINEN,TUIJA",31,2017-09-08,PUHELINKONTAKTI PERUSTYÖAIKANA,66.0,True,65650030,ABA1A,...,True,Phone,NaN,NaN,Other diseases of the musculoskeletal system,None,NaN,NaN,NaN,None


In [21]:
contacts.head()

,First visit,Kiireellisyys,Luonne,Tulos,Toimipiste,Patient ID,Tila,Yhteydenotto-Tapahtuma,Kello,Time,Yo-av vastaanottaja,Yo-av käyntityyppi,Yo-av ammattiryhmän nimi,Yo-av päivä,Yhteydenoton lisätieto,Yo-av lkm,Yhteydenotto aika,Cost,Klinik,Phone
0,False,NaN,NaN,NaN,Myyrmäen terveysasema A-tiimi,65650030,1,<NA>,11:56,2017-11-23,NaN,NaN,NaN,NaT,<NA>,<NA>,2017-11-23 11:56:00,0.0,False,False
1,False,Ei-kiireellinen hoidon tarve,Sairaanhoito,Ajanvaraus,Myyrmäen terveysasema A-tiimi,65650030,3,"ääni on käheä, yskää",12:47,2018-08-21,NaN,NaN,NaN,NaT,<NA>,<NA>,2018-08-21 12:47:00,0.0,False,False
2,False,NaN,NaN,NaN,Myyrmäen terveysasema B-tiimi,65650038,1,<NA>,12:04,2017-09-11,NaN,NaN,NaN,NaT,RES UUSINTAPYYNTÖ L.VALIOLLE,<NA>,2017-09-11 12:04:00,0.0,False,False
3,False,Ei-kiireellinen hoidon tarve,Sairaanhoito,Ajanvaraus,Myyrmäen terveysasema A-tiimi,65650038,3,mmse,10:18,2017-11-15,NaN,NaN,NaN,NaT,<NA>,<NA>,2017-11-15 10:18:00,0.0,False,False
4,False,NaN,NaN,NaN,Myyrmäen terveysasema,65650038,1,<NA>,11:33,2018-06-15,NaN,NaN,NaN,NaT,Ajan tarkistus,<NA>,2018-06-15 11:33:00,0.0,False,False


In [22]:
bookings.head()

,Ajanvaraus-Päivä,Ajanvaraus-Klo,Käyntityyppi,Vastaanottaja,Sähköinen-Asiointi,Toimipiste lyh,Toimipisteen nimi,Ajanvaraus-Tallennusaika,Patient ID,Varattu aika,Ajanvaraus-Tallennuspäivä,Waiting time,Cost
0,2017-08-17,09:00:00,FSV,"SALMINEN,MERVI",False,MYY,Myyrmäen terveysasema,2017-08-16 12:03:14,66795523,2017-08-17 09:00:00,2017-08-16,0.872755,0.0
1,2017-08-24,09:00:00,FSV,"SALMINEN,MERVI",False,MYY,Myyrmäen terveysasema,2017-08-23 09:15:11,65659915,2017-08-24 09:00:00,2017-08-23,0.989456,0.0
2,2017-08-24,10:15:00,FSV,"SALMINEN,MERVI",False,MYY,Myyrmäen terveysasema,2017-08-24 08:18:09,65808324,2017-08-24 10:15:00,2017-08-24,0.081146,0.0
3,2017-08-31,09:00:00,FSV,"SALMINEN,MERVI",False,MYY,Myyrmäen terveysasema,2017-08-31 08:20:41,66790760,2017-08-31 09:00:00,2017-08-31,0.027303,0.0
4,2017-09-14,09:00:00,FSV,"SALMINEN,MERVI",False,MYY,Myyrmäen terveysasema,2017-09-12 12:44:36,65666405,2017-09-14 09:00:00,2017-09-12,1.844028,0.0
